# Lesson  3 Building the Tutor Service Layer

## Building the Tutor Service Layer

## Implementing Session Creation in the Tutor Service

## Implementing Query Processing for Tutor Service

## Implement AI Response Generation with DeepSeek in TutorService

## Testing Context Retention in AI Tutor Conversations

## Creating Multiple Tutoring Sessions for a Student